In [ ]:
# ==========================================================
# Libraries
# ==========================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import scipy.stats as stats

# ==========================================================
# Load and prepare MASI data
# ==========================================================
df = pd.read_csv("masi.csv")
df["Date"] = pd.to_datetime(df["Date"])

# Sort ascending (past -> present)
df = df.sort_values("Date").reset_index(drop=True)

# Keep only Date + Opening Price
masi = df[["Date", "Open"]].copy()
masi.rename(columns={"Open": "OpenPrice"}, inplace=True)

# Ensure numeric (strip commas if present)
masi["OpenPrice"] = (
    masi["OpenPrice"].astype(str).str.replace(",", "", regex=False).astype(float)
)

# Log transform and first-order differencing
masi["LogOpen"] = np.log1p(masi["OpenPrice"])
masi["Diff1_LogOpen"] = masi["LogOpen"].diff()
diff_series = masi.dropna(subset=["Diff1_LogOpen"])

# Style variables
outer_bg = "#cce6ff"   # light blue background
inner_bg = "white"     # white plot panels

# ==========================================================
# 1. Original MASI opening price (time series)
# ==========================================================
fig, ax = plt.subplots(figsize=(12,6))
fig.patch.set_facecolor(outer_bg)
ax.set_facecolor(inner_bg)

ax.plot(masi["Date"], masi["OpenPrice"], color="blue", linewidth=1.5,
        label="MASI Opening Price (MAD)")

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
plt.xticks(rotation=45)

ax.set_xlabel("Date")
ax.set_ylabel("Opening Price (MAD)")
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("masi_timeseries.png", dpi=300, facecolor=fig.get_facecolor())
plt.show()

# ==========================================================
# 2. Time series with ARIMA(2,1,7) fitted values + 99% CI
# ==========================================================
# Fit ARIMA on log scale with first differencing
model = ARIMA(masi["LogOpen"], order=(2,1,7))
fit = model.fit()

# Drop burn-in observations (where ARIMA is unstable)
start_idx = fit.loglikelihood_burn
pred = fit.get_prediction(start=start_idx, end=len(masi)-1)
pred_mean_log = pred.predicted_mean
pred_ci_log = pred.conf_int(alpha=0.01)  # 99% CI

# Back-transform to original scale
pred_mean = np.expm1(pred_mean_log)
pred_ci = np.expm1(pred_ci_log)

fig, ax = plt.subplots(figsize=(12,6))
fig.patch.set_facecolor(outer_bg)
ax.set_facecolor(inner_bg)

ax.plot(masi["Date"], masi["OpenPrice"], color="blue", linewidth=1.5,
        label="Observed opening price")
ax.plot(masi["Date"].iloc[start_idx:], pred_mean, color="black", linewidth=1.2,
        label="ARIMA(2,1,7) fitted values")
ax.fill_between(masi["Date"].iloc[start_idx:], pred_ci.iloc[:,0], pred_ci.iloc[:,1],
                color="blue", alpha=0.2, label="99% CI")

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
plt.xticks(rotation=45)

ax.set_xlabel("Date")
ax.set_ylabel("Opening Price (MAD)")
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("masi_with_ci.png", dpi=300, facecolor=fig.get_facecolor())
plt.show()

# ==========================================================
# 3. First-order differenced log-transformed series
# ==========================================================
fig, ax = plt.subplots(figsize=(12,6))
fig.patch.set_facecolor(outer_bg)
ax.set_facecolor(inner_bg)

ax.plot(diff_series["Date"], diff_series["Diff1_LogOpen"], color="blue", linewidth=1.5,
        label="First-order differenced log opening price")
ax.axhline(0, color="gray", linestyle="--")

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
plt.xticks(rotation=45)

ax.set_xlabel("Date")
ax.set_ylabel("First-order differenced log(OpenPrice)")
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("masi_first_order_diff.png", dpi=300, facecolor=fig.get_facecolor())
plt.show()

# ==========================================================
# 4. ACF and PACF plots (99% CI, lag=30)
# ==========================================================
fig, axes = plt.subplots(2, 1, figsize=(10,8))
fig.patch.set_facecolor(outer_bg)

for ax in axes:
    ax.set_facecolor(inner_bg)

plot_acf(diff_series["Diff1_LogOpen"].dropna(), lags=30, ax=axes[0], alpha=0.01)
axes[0].set_ylabel("Autocorrelation")
axes[0].set_xlabel("Lag")
axes[0].legend(["ACF (99% CI shaded)"])

plot_pacf(diff_series["Diff1_LogOpen"].dropna(), lags=30, ax=axes[1], alpha=0.01, method="ywm")
axes[1].set_ylabel("Partial autocorrelation")
axes[1].set_xlabel("Lag")
axes[1].legend(["PACF (99% CI shaded)"])

plt.tight_layout()
plt.savefig("masi_acf_pacf.png", dpi=300, facecolor=fig.get_facecolor())
plt.show()

# ==========================================================
# 5. Residual diagnostics (2x2 panel) with cleaned residuals
# ==========================================================
residuals = fit.resid[start_idx:]  # drop unstable early values
residuals_clean = residuals[np.abs(residuals) < 1]  # remove outliers like 9.41
fitted_vals = fit.fittedvalues[start_idx:]

fig, axes = plt.subplots(2, 2, figsize=(12,10))
fig.patch.set_facecolor("#f5f5dc")

for ax in axes.flatten():
    ax.set_facecolor("white")

# --- Q-Q Plot ---
osm, osr = stats.probplot(residuals_clean, dist="norm")[:2]
theoretical_q = osm[0]
sample_q = osm[1]
axes[0,0].scatter(theoretical_q, sample_q, color="red", s=30)
slope, intercept, *_ = stats.linregress(theoretical_q, sample_q)
axes[0,0].plot(theoretical_q, intercept + slope*theoretical_q, color="blue")
axes[0,0].set_xlim(-3, 3)
axes[0,0].set_ylim(-0.4, 0.4)
axes[0,0].set_xlabel("Theoretical quantiles")
axes[0,0].set_ylabel("Sample quantiles")
axes[0,0].set_title("Normal Probability Plot")

# --- Residuals vs Fitted ---
axes[0,1].scatter(fitted_vals, residuals_clean, color="red", s=30)
axes[0,1].axhline(0, color="black", linestyle="--")
axes[0,1].set_xlabel("Fitted values")
axes[0,1].set_ylabel("Residuals")
axes[0,1].set_title("Residuals vs Fitted")

# --- Histogram + KDE ---
sns.histplot(residuals_clean, bins=20, color="gray", stat="density", ax=axes[1,0])
sns.kdeplot(residuals_clean, color="blue", ax=axes[1,0], linewidth=2)
axes[1,0].set_xlim(-0.1, 0.1)
axes[1,0].set_xlabel("Residuals")
axes[1,0].set_ylabel("Density")
axes[1,0].set_title("Histogram of Residuals")

# --- Residuals vs Order ---
axes[1,1].plot(range(len(residuals_clean)), residuals_clean, color="blue")
axes[1,1].axhline(0, color="black", linestyle="--")
axes[1,1].set_xlim(-10, 600)   # add padding before 0
axes[1,1].set_ylim(-0.1, 0.1)
axes[1,1].set_xlabel("Observation order")
axes[1,1].set_ylabel("Residuals")
axes[1,1].set_title("Residuals vs Order")

plt.tight_layout()
plt.savefig("masi_residual_diagnostics.png", dpi=300, facecolor=fig.get_facecolor())
plt.show()
